In [ ]:
!pip install -q keras-nlp


In [ ]:
import keras_nlp
import tensorflow as tf
from tensorflow import keras
import time

Using TensorFlow backend


In [ ]:
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

In [ ]:
start = time.time()

output = gpt2_lm.generate("I suck at expressing emotions", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
I suck at expressing emotions. I'm not sure I have the right words for it, but that's what makes me a better listener. I'm a person who's been through a lot.

I've always had this feeling of being alone in my life, and I'm not sure if that's true anymore. I'm not sure I want to be alone, but I don't want to have that feeling of being alone in my life anymore.

I've never been alone in my life.

But, I've been lonely for the last 10 years. I've been lonely for the last 20 years. And I've been lonely because I've been a part of it.

And that's what I want to do. That's what I want to do.

I want to be alone in my life, because that's what I want to do.

I want to be alone in my life, because that's what I want to do
TOTAL TIME ELAPSED: 60.22s


In [ ]:
start = time.time()

output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
That Italian restaurant is the most famous in the world. It is the oldest in the country and the oldest Italian restaurant in Italy. It's the oldest restaurant in the world and the oldest restaurant in Italy. It's the oldest restaurant in the world. That Italian restaurant is the most famous in the world. It is the oldest in the country and the oldest in Italy. It's the oldest in the world. It's the oldest in the world.

That Italian restaurant is the most famous in the world. It is the oldest in the country and the oldest in Italy. It's the oldest in the world. It's the oldest in the world. That Italian restaurant is the most famous in the country. It is the oldest in the country and the oldest in Italy. It's the oldest in the world. It's the oldest in the world.

That Italian restaurant is the most famous in the world. It is the oldest in the country and the oldest in Italy. It
TOTAL TIME ELAPSED: 40.58s


**Finetune on Reddit dataset**

In [ ]:
import tensorflow_datasets as tfds

reddit_ds = tfds.load("reddit_tifu", split="train", as_supervised=True)

In [ ]:
for document, title in reddit_ds:
    print(document.numpy())
    print(title.numpy())
    break

b"me and a friend decided to go to the beach last sunday. we loaded up and headed out. we were about half way there when i decided that i was not leaving till i had seafood. \n\nnow i'm not talking about red lobster. no friends i'm talking about a low country boil. i found the restaurant and got directions. i don't know if any of you have heard about the crab shack on tybee island but let me tell you it's worth it. \n\nwe arrived and was seated quickly. we decided to get a seafood sampler for two and split it. the waitress bought it out on separate platters for us. the amount of food was staggering. two types of crab, shrimp, mussels, crawfish, andouille sausage, red potatoes, and corn on the cob. i managed to finish it and some of my friends crawfish and mussels. it was a day to be a fat ass. we finished paid for our food and headed to the beach. \n\nfunny thing about seafood. it runs through me faster than a kenyan \n\nwe arrived and walked around a bit. it was about 45min since we a

In [ ]:
train_ds = (
    reddit_ds.map(lambda document, _: document)
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
train_ds = train_ds.take(200)
num_epochs = 1

# Linearly decaying learning rate.
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

200/200 [==============================] - 14527s 72s/step - loss: 3.3625 - accuracy: 0.3188


In [ ]:
start = time.time()

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
I like basketball. i'm a big kid, a big boy with a great sense of humor and a good sense of humor about it. i'm a big guy, and i like being a big boy.

I'm a little older than my sister, but i still like to play basketball.

i'm a big kid, but i still like to play basketball. i'm a little older than my sisters, but i still like to play basketball. my sister's basketball was a big game, but she was also very competitive.
TOTAL TIME ELAPSED: 41.43s
